In [88]:
from datetime import date, timedelta

class Daycount():
    
    """
    Rules:
        The formulas below use the ISDA 2006 rules
        https://www.isda.org/a/smMDE/Blackline-2000-v-2006-ISDA-Definitions.pdf
        Section 4.16
    
    Parameters:
        start_date (datetime object): beginning of period
        end_date (datetime object): end of period
        numerator_type (string): the numerator calc method for the daycount fraction
            'actual': actual number of days from start_date to end_date
            '30': number of days from start_date to end_date using 30/360 definition
        denominator_type (string): the denominator calc method for the daycount fraction
            '360': use 360 days in denominator (common for money market rates)
            '365': use 365 days in denominator 
            'actual': use actual number of days in denominator (366 for leap year and 365 otherwise)
    """
        
    def __init__(self):
        # constructor 
        pass

    
    def daycount_fraction(self, start_date, end_date, numerator_type, denominator_type):
        start_date = start_date.date()
        end_date = end_date.date()
        
        answer = 'error'
        
        if denominator_type == '360':
            numerator = self.calc_numerator(start_date, end_date, numerator_type) 
            if type(numerator) != str:
                    answer = numerator / 360
        elif denominator_type == '365':
            if numerator_type == 'actual':
                numerator = self.calc_numerator(start_date, end_date, numerator_type) 
                if type(numerator) != str:
                    answer = numerator / 365
        elif denominator_type == 'actual':
            if numerator_type == 'actual':
                initial_answer = self.calc_act_act_fraction(start_date, end_date)
                if type(initial_answer) != str:
                    answer = initial_answer
        return answer

    
    def calc_numerator(self, start_date, end_date, numerator_type):
        if numerator_type == 'actual':
            answer = self.actual_numerator(start_date, end_date)
        elif numerator_type == '30':
            answer = self.thirty_numerator(start_date, end_date)
        else:
            answer = 'error'
        return answer
            
        
    def actual_numerator(self, start_date, end_date):
        answer = (end_date - start_date).days 
        return answer
    
    
    def thirty_numerator(self, start_date, end_date):
        y1 = start_date.year
        y2 = end_date.year
        days = 360 * (y2 - y1)
        
        m1 = start_date.month
        m2 = end_date.month
        days = days + 30 * (m2 - m1)
    
        if d1 == 31: # Adjustment per ISDA 30/360 rules
            d1 = 30
        d2 = end_date.day
        if d2 == 31 and d1 > 29: # Adjustment per ISDA 30/360 rules
            d2 = 30
        days = days + (d2 - d1)
        
        return days
    
    
    def calc_act_act_fraction(self, start_date, end_date):
        answer = 0.0
        
        if start_date <= end_date:

            current_date = start_date

            while current_date < end_date:
                year_end = date(current_date.year, 12, 31)
                current_end = min(year_end, end_date)

                days_in_period = (current_end - current_date).days + 1
                days_in_year = 366 if self.is_leap_year(current_date.year) else 365
                
                answer = answer + days_in_period / days_in_year

                current_date = current_end + timedelta(days=1)

        return answer

    
    def is_leap_year(self, year):
        # Return True if year is a leap year
        return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
    